## Choosing a pre training model
The model used for this project is `ssd_mobilenet_v2_coco`.
Check other models from [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models).

Because the interestes of this project is to interfere on real time video, i am chosing a model that has a high inference speed `(ms)` with relativly high `mAP` on COCO

In [1]:

%tensorflow_version 1.x # Select module of the tensorflow
# Some models to train on
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
    }
}

# Select a model in `MODELS_CONFIG`.
# I chose ssd_mobilenet_v2 for this project, you could choose any
selected_model = 'ssd_mobilenet_v2'


`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x # Select module of the tensorflow`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


## Installing Required Packages 

In [2]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -qq Cython contextlib2 pillow lxml matplotlib

!pip install -qq pycocotools

Selecting previously unselected package python-bs4.
(Reading database ... 144579 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpacking pyt

## General imports
Other Imports will be done after downloading some packages later.

In [3]:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv
import re
import cv2 
import os
import glob
import xml.etree.ElementTree as ET

import io
import tensorflow.compat.v1 as tf

from PIL import Image
from collections import namedtuple, OrderedDict

import shutil
import urllib.request
import tarfile

from google.colab import files

In [4]:
#we need tenorflow v 1.15.0, object detection API is removed from tf v 2.0+
print(tf.__version__)

1.15.2


## Downloading and Orgniazing Images and Annotations
1. Downloading the images and annotations from the [source](https://sci2s.ugr.es/weapons-detection)  and unziping them
2. Creating a directory `(data)` to save some data such as; images, annotation, csv, etc...
3. Creating two directories; for the training and testing labels (not the images)
4. Randomly splitting our labels into 80% training and 20% testing and moving the splits to their directories: `(train_labels)` & `(test_labels)` 

In [5]:
#creates a directory for the whole project
!mkdir gun_detection

In [6]:
!git clone https://github.com/Dovahkiin169/CareLabelsData.git

Cloning into 'CareLabelsData'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 565 (delta 116), reused 114 (delta 97), pack-reused 432
Receiving objects: 100% (565/565), 190.23 MiB | 14.02 MiB/s, done.
Resolving deltas: 100% (328/328), done.


In [7]:
cd gun_detection

/content/gun_detection


## Preprocessing Images and Labels
1. Converting the annotations from xml files to two csv files for each `train_labels/` and `train_labels/`.
2. Creating a pbtxt file that specifies the number of class (one class in this case)
3. Checking if the annotations for each object are placed within the range of the image width and height.

In [8]:

#adjusted from: https://github.com/datitran/raccoon_dataset

#converts the annotations/labels into one csv file for each training and testing labels
#creats label_map.pbtxt file

%cd /content/gun_detection/data


# images extension
images_extension = 'jpg'

# takes the path of a directory that contains xml files and converts
#  them to one csv file.

# returns a csv file that contains: image name, width, height, class, xmin, ymin, xmax, ymax.
# note: if the xml file contains more than one box/label, it will create more than one row for the same image. each row contains the info for an individual box. 
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text + '.' + images_extension,
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

# for both the train_labels and test_labels csv files, it runs the xml_to_csv() above.
for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

# Creating the `label_map.pbtxt` file
label_map_path = os.path.join("label_map.pbtxt")

pbtxt_content = ""

#creats a pbtxt file the has the class names.
for i, class_name in enumerate(classes):
    # display_name is optional.
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n    display_name: '{1}'\n }}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)


/content/gun_detection/data
Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.


In [10]:
#checking the pbtxt file
!cat label_map.pbtxt

item {
    id: 1
    name: 'bleaching_with_chlorine_allowed'
    display_name: 'bleaching_with_chlorine_allowed'
 }

 item {
    id: 2
    name: 'chlorine_and_non_chlorine_bleach'
    display_name: 'chlorine_and_non_chlorine_bleach'
 }

 item {
    id: 3
    name: 'do_not_bleach'
    display_name: 'do_not_bleach'
 }

 item {
    id: 4
    name: 'do_not_dry_clean'
    display_name: 'do_not_dry_clean'
 }

  item {
    id: 5
    name: 'do_not_iron'
    display_name: 'do_not_iron'
 }

  item {
    id: 6
    name: 'do_not_tumble_drying'
    display_name: 'do_not_tumble_drying'
 }

  item {
    id: 7
    name: 'do_not_wash'
    display_name: 'do_not_wash'
 }

  item {
    id: 8
    name: 'do_not_wet_clean'
    display_name: 'do_not_wet_clean'
 }

  item {
    id: 9
    name: 'drip_dry_in_the_shade'
    display_name: 'drip_dry_in_the_shade'
 }

  item {
    id: 10
    name: 'drip_dry'
    display_name: 'drip_dry'
 }

  item {
    id: 11
    name: 'dry_clean_any_solvent'
    display_name: 'dry

## Downloading and Preparing Tensorflow model
1. Cloning [Tensorflow models](https://github.com/tensorflow/models.git) from the offical git repo. The repo contains the object detection API we are interseted in. 
2. Compiling the protos and adding folders to the os environment.
3. Testing the model builder.

In [11]:
# Downlaods Tenorflow
%cd /content/gun_detection/
!git clone --q https://github.com/tensorflow/models.git

/content/gun_detection


In [12]:
%cd /content/gun_detection/models/research
#compiling the proto buffers (not important to understand for this project but you can learn more about them here: https://developers.google.com/protocol-buffers/)
!protoc object_detection/protos/*.proto --python_out=.

# exports the PYTHONPATH environment variable with the reasearch and slim folders' paths
os.environ['PYTHONPATH'] += ':/content/gun_detection/models/research/:/content/gun_detection/models/research/slim/'

/content/gun_detection/models/research


In [13]:
# testing the model builder
!pip3 install tf_slim
!python3 object_detection/builders/model_builder_test.py

     |████████████████████████████████| 358kB 2.8MB/s 


## Generating Tf record
- Generating two TFRecords files for the training and testing CSVs.
- Tensorflow accepts the data as tfrecords which is a binary file that run fast with low memory usage. Instead of loading the full data into memory, Tenorflow breaks the data into batches using these TFRecords automatically

In [14]:
#adjusted from: https://github.com/datitran/raccoon_dataset

# converts the csv files for training and testing data to two TFRecords files.
# places the output in the same directory as the input

from object_detection.utils import dataset_util
%cd /content/gun_detection/models/

DATA_BASE_PATH = '/content/gun_detection/data/'
image_dir = '/content/gun_detection/data/images'

def class_text_to_int(row_label):
		if row_label == 'bleaching_with_chlorine_allowed':
				return 1
		elif row_label == 'chlorine_and_non_chlorine_bleach':
				return 2
		elif row_label == 'do_not_bleach':
				return 3
		elif row_label == 'do_not_dry_clean':
				return 4
		elif row_label == 'do_not_iron':
				return 5
		elif row_label == 'do_not_tumble_drying':
				return 6
		elif row_label == 'do_not_wash':
				return 7
		elif row_label == 'do_not_wet_clean':
				return 8
		elif row_label == 'drip_dry_in_the_shade':
				return 9
		elif row_label == 'drip_dry':
				return 10
		elif row_label == 'dry_clean_any_solvent':
				return 11
		elif row_label == 'dry_clean_hydrocarbon_solvent_only_hcs':
				return 12
		elif row_label == 'dry_clean_tetrachloroethylene_pce_only':
				return 13
		elif row_label == 'dry_flat_in_the_shade':
				return 14
		elif row_label == 'dry_flat':
				return 15
		elif row_label == 'dry_in_the_shade':
				return 16
		elif row_label == 'drying_symbol':
				return 17
		elif row_label == 'gentle_cleaning_with_hydrocarbon_solvents':
				return 18
		elif row_label == 'gentle_cleaning_with_pce':
				return 19
		elif row_label == 'gentle_wet_cleaning':
				return 20
		elif row_label == 'hand_wash':
				return 21
		elif row_label == 'ironing_at_high_temp':
				return 22
		elif row_label == 'ironing_at_low_temp':
				return 23
		elif row_label == 'ironing_at_med_temp':
				return 24
		elif row_label == 'ironing':
				return 25
		elif row_label == 'line_dry_in_the_shade':
				return 26
		elif row_label == 'line_dry':
				return 27
		elif row_label == 'no_steam':
				return 28
		elif row_label == 'non_chlorine_bleach_when_needed':
				return 29
		elif row_label == 'professional_cleaning':
				return 30
		elif row_label == 'professional_wet_cleaning':
				return 31
		elif row_label == 'tumble_drying_low_temps':
				return 32
		elif row_label == 'tumble_drying_normal':
				return 33
		elif row_label == 'tumble_drying':
				return 34
		elif row_label == 'very_gentle_cleaning_with_hydrocarbon_solvents':
				return 35
		elif row_label == 'very_gentle_cleaning_with_pce':
				return 36
		elif row_label == 'very_gentle_wet_cleaning':
				return 37
		elif row_label == 'wash_at_or_below_30_mild_fine_wash':
				return 38
		elif row_label == 'wash_at_or_below_30_very_mild_fine_wash':
				return 39
		elif row_label == 'wash_at_or_below_30':
				return 40
		elif row_label == 'wash_at_or_below_40_mild_fine_wash':
				return 41
		elif row_label == 'wash_at_or_below_40_very_mild_fine_wash':
				return 42
		elif row_label == 'wash_at_or_below_40':
				return 43
		elif row_label == 'wash_at_or_below_50_mild_fine_wash':
				return 44
		elif row_label == 'wash_at_or_below_50':
				return 45
		elif row_label == 'wash_at_or_below_60_mild_fine_wash':
				return 46
		elif row_label == 'wash_at_or_below_60':
				return 47
		elif row_label == 'wash_at_or_below_70':
				return 48
		elif row_label == 'wash_at_or_below_90':
				return 49
		elif row_label == 'washing_symbol':
				return 50			
		else:
				0

def split(df, group):
		data = namedtuple('data', ['filename', 'object'])
		gb = df.groupby(group)
		return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
		with tf.io.gfile.GFile(os.path.join(path, '{}'.format(os.path.splitext(group.filename)[0])), 'rb') as fid:
				encoded_jpg = fid.read()
		encoded_jpg_io = io.BytesIO(encoded_jpg)
		image = Image.open(encoded_jpg_io)
		width, height = image.size

		filename = group.filename.encode('utf8')
		#image_format = 'jpg'
		xmins = []
		xmaxs = []
		ymins = []
		ymaxs = []
		classes_text = []
		classes = []

		for index, row in group.object.iterrows():
				xmins.append(row['xmin'] / width)
				xmaxs.append(row['xmax'] / width)
				ymins.append(row['ymin'] / height)
				ymaxs.append(row['ymax'] / height)
				classes_text.append(row['class'].encode('utf8'))
				classes.append(class_text_to_int(row['class']))

		tf_example = tf.train.Example(features=tf.train.Features(feature={
				'image/height': dataset_util.int64_feature(height),
				'image/width': dataset_util.int64_feature(width),
				'image/filename': dataset_util.bytes_feature(os.path.splitext(filename)[0]),
				'image/source_id': dataset_util.bytes_feature(os.path.splitext(filename)[0]),
				'image/encoded': dataset_util.bytes_feature(encoded_jpg),
				'image/format': dataset_util.bytes_feature(b'jpg'),
				'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
				'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
				'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
				'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
				'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
				'image/object/class/label': dataset_util.int64_list_feature(classes),
		}))
		print(classes)
		return tf_example

for csv in ['train_labels', 'test_labels']:
  writer = tf.io.TFRecordWriter(DATA_BASE_PATH + csv + '.record')
  path = os.path.join(image_dir)
  examples = pd.read_csv(DATA_BASE_PATH + csv + '.csv')
  grouped = split(examples, 'filename')
  for group in grouped:
      tf_example = create_tf_example(group, path)
      writer.write(tf_example.SerializeToString())
    
  writer.close()
  output_path = os.path.join(os.getcwd(), DATA_BASE_PATH + csv + '.record')
  print('Successfully created the TFRecords: {}'.format(DATA_BASE_PATH +csv + '.record'))


/content/gun_detection/models
[43, 1, 32, 24, 4]
[38, 38, 43, 1, 32, 24, 4]
[43, 1, 6, 24, 4]
[40, 1, 6, 24, 4]
[40, 40, 1, 1, 40, 1, 6, 6, 23, 23, 13, 13, 6]
[40, 1, 6, 23, 4]
[38, 1, 6, 23, 4]
[43, 1, 32, 24, 4]
[40, 1, 32, 24, 4]
[43, 1, 32, 23, 4]
[40, 1, 6, 23, 4]
[41, 1, 6, 24, 13]
[38, 38, 14, 1, 1, 6, 6, 23, 23, 13, 13]
[38, 1, 6, 23, 4]
[43, 1, 32, 24, 4]
[23, 13, 32]
[43, 1, 6, 24, 13, 27]
[43, 24, 4, 1, 6]
[43, 1, 32, 24, 4]
[5, 32, 38, 4, 1]
[21, 6, 15, 24, 1, 4]
[23, 40, 1, 6, 13]
[43, 1, 32, 23, 13]
[38, 1, 6, 23, 4]
[38, 38, 14, 1, 1, 6, 6, 23, 23, 13, 13]
[41, 41, 1, 1, 6, 6, 24, 24, 13, 13]
[43, 1, 32, 24, 4]
[41, 1, 24, 13, 6]
[40, 1, 6, 27, 24, 4]
[40, 1, 6, 24, 4]
[43, 1, 32, 24, 4, 40]
[42, 1, 6, 23, 4]
[40, 40, 1, 1, 32, 32, 24, 24, 13, 13]
[43, 1, 32, 24, 4, 40, 40]
[7, 1, 23, 6, 13]
[21, 1, 24, 4, 6, 15]
[40, 40, 40, 29, 29, 29, 32, 32, 32, 23, 23, 23]
[40, 1, 6, 23, 23]
[40, 40, 40, 1, 1, 1, 1, 6, 6, 6, 23, 23, 13, 13]
[7, 1, 5, 6, 19]
[21, 6, 15, 24, 1, 4]
[40

In [15]:
# TFRecords are created
!ls -lX /content/gun_detection/data/

total 66792
drwxr-xr-x 2 root root     4096 Sep 11 15:11 images
drwxr-xr-x 2 root root     4096 Sep 11 15:11 test_labels
drwxr-xr-x 2 root root     4096 Sep 11 15:11 train_labels
-rw-r--r-- 1 root root      929 Sep 11 15:12 test_labels.csv
-rw-r--r-- 1 root root    32735 Sep 11 15:12 train_labels.csv
-rw-r--r-- 1 root root     5092 Sep 11 15:12 label_map.pbtxt
-rw-r--r-- 1 root root  2006675 Sep 11 15:13 test_labels.record
-rw-r--r-- 1 root root 66329466 Sep 11 15:13 train_labels.record


## Downloading the Base Model
1. Based on the model selecting at the top of this notebook, downloading the model selected and extracting its content.
2. Creating a dir to save the model while training.

In [16]:
%cd /content/gun_detection/models/research

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

#selecting the model
MODEL_FILE = MODEL + '.tar.gz'

#creating the downlaod link for the model selected
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

#the distination folder where the model will be saved
fine_tune_dir = '/content/gun_detection/models/research/pretrained_model'

#checks if the model has already been downloaded
if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

#unzipping the file and extracting its content
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# creating an output file to save the model while training
os.remove(MODEL_FILE)
if (os.path.exists(fine_tune_dir)):
    shutil.rmtree(fine_tune_dir)
os.rename(MODEL, fine_tune_dir)

/content/gun_detection/models/research


In [20]:
#checking the content of the pretrained model.
# this is the directory of the "fine_tune_checkpoint" that is used in the config file.
!echo {fine_tune_dir}
!ls -alh {fine_tune_dir}

/content/gun_detection/models/research/pretrained_model
total 135M
drwxr-xr-x  3 345018 89939 4.0K Mar 30  2018 .
drwxr-xr-x 24 root   root  4.0K Sep 11 15:13 ..
-rw-r--r--  1 345018 89939   77 Mar 30  2018 checkpoint
-rw-r--r--  1 345018 89939  67M Mar 30  2018 frozen_inference_graph.pb
-rw-r--r--  1 345018 89939  65M Mar 30  2018 model.ckpt.data-00000-of-00001
-rw-r--r--  1 345018 89939  15K Mar 30  2018 model.ckpt.index
-rw-r--r--  1 345018 89939 3.4M Mar 30  2018 model.ckpt.meta
-rw-r--r--  1 345018 89939 4.2K Mar 30  2018 pipeline.config
drwxr-xr-x  3 345018 89939 4.0K Mar 30  2018 saved_model


## Configuring the Training Pipeline
1. Adding the path for the TFRecords files and pbtxt,batch_size,num_steps,num_classes to the configuration file.
2. Adding some Image augmentation.
3. Creating a directory to save the model at each checkpoint while training. 

In [21]:

#the path to the folder containing all the sample config files
CONFIG_BASE = "/content/gun_detection/models/research/object_detection/samples/configs/"

#path to the specified model's config file
model_pipline = os.path.join(CONFIG_BASE, pipeline_file)
model_pipline

'/content/gun_detection/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config'

In [19]:
#check the sample config file that is provided by the tf model
#!cat /content/gun_detection/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config

In [22]:
#editing the configuration file to add the path for the TFRecords files, pbtxt,batch_size,num_steps,num_classes.
# any image augmentation, hyperparemeter tunning (drop out, batch normalization... etc) would be editted here

%%writefile {model_pipline}
model {
  ssd {
    num_classes: 1 # number of classes to be detected
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 3.0
        aspect_ratios: 0.3333
      }
    }
    # all images will be resized to the below W x H.
    image_resizer { 
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    box_predictor {
      convolutional_box_predictor {
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        #use_dropout: false
        use_dropout: true # to counter over fitting. you can also try tweaking its probability below
        dropout_keep_probability: 0.8
        kernel_size: 1
        box_code_size: 4
        apply_sigmoid_to_scores: false
        conv_hyperparams {
          activation: RELU_6,
          regularizer {
            l2_regularizer {
            # weight: 0.00004
            weight: 0.001 # higher regularizition to counter overfitting
          }
          }
          initializer {
            truncated_normal_initializer {
              stddev: 0.03
              mean: 0.0
            }
          }
          batch_norm {
            train: true,
            scale: true,
            center: true,
            decay: 0.9997,
            epsilon: 0.001,
          }
        }
      }
    }
    feature_extractor {
      type: 'ssd_mobilenet_v2'
      min_depth: 16
      depth_multiplier: 1.0
      conv_hyperparams {
        activation: RELU_6,
        regularizer {
          l2_regularizer {
            # weight: 0.00004
            weight: 0.001 # higher regularizition to counter overfitting
          }
        }
        initializer {
          truncated_normal_initializer {
            stddev: 0.03
            mean: 0.0
          }
        }
        batch_norm {
          train: true,
          scale: true,
          center: true,
          decay: 0.9997,
          epsilon: 0.001,
        }
      }
    }
    loss {
      classification_loss {
        weighted_sigmoid {
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      hard_example_miner {
        num_hard_examples: 3000 
        iou_threshold: 0.95
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 3
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    normalize_loss_by_num_matches: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.6
        
        #adjust this to the max number of objects per class. 
        # ex, in my case, i have one pistol in most of the images.
        # . there are some images with more than one up to 16.
        max_detections_per_class: 16
        # max number of detections among all classes. I have 1 class only so
        max_total_detections: 16
      }
      score_converter: SIGMOID
    }
  }
}

train_config: {
  batch_size: 16 # training batch size
  optimizer {
    rms_prop_optimizer: {
      learning_rate: {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.003
          decay_steps: 800720
          decay_factor: 0.95
        }
      }
      momentum_optimizer_value: 0.9
      decay: 0.9
      epsilon: 1.0
    }
  }

  #the path to the pretrained model. 
  fine_tune_checkpoint: "/content/gun_detection/models/research/pretrained_model/model.ckpt"
  fine_tune_checkpoint_type:  "detection"
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 1000 
  

  #data augmentaion is done here, you can remove or add more.
  # They will help the model generalize but the training time will increase greatly by using more data augmentation.
  # Check this link to add more image augmentation: https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto
  
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_adjust_contrast {
    }
  }
  data_augmentation_options {
    ssd_random_crop {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    #path to the training TFRecord
    input_path: "/content/gun_detection/data/train_labels.record"
  }
  #path to the label map 
  label_map_path: "/content/gun_detection/data/label_map.pbtxt"
}

eval_config: {
  # the number of images in your "testing" data (was 600 but we removed one above :) )
  num_examples: 599
  # the number of images to disply in Tensorboard while training
  num_visualizations: 20

  # Note: The below line limits the evaluation process to 10 evaluations.
  # Remove the below line to evaluate indefinitely.
  #max_evals: 10
}

eval_input_reader: {
  tf_record_input_reader {
      
    #path to the testing TFRecord
    input_path: "/content/gun_detection/data/test_labels.record"
  }
  #path to the label map 
  label_map_path: "/content/gun_detection/data/label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

Overwriting /content/gun_detection/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config


In [23]:
# where the model will be saved at each checkpoint while training 
model_dir = 'training/'

# Optionally: remove content in output model directory to fresh start.
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

## Tensorboard
1. Downlaoding and unzipping Tensorboard
2. creating a link to visualize multiple graph while training.


notes: 
  1. Tensorboard will not log any files until the training starts. 
  2. a max of 20 connection per minute is allowed when using ngrok, you will not be able to access tensorboard while the model is logging.

In [24]:
#downlaoding ngrok to be able to access tensorboard on google colab
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2020-09-11 15:14:13--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.44.17.83, 52.205.91.57, 52.54.124.219, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.44.17.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  6.45MB/s    in 2.0s    

2020-09-11 15:14:16 (6.45 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [25]:
#the logs that are created while training 
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')

In [26]:
#The link to tensorboard.
#works after the training starts.

### note: if you didnt get a link as output, rerun this cell and the one above
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://a84b35031134.ngrok.io


## Training

Finally training the model!


In [ ]:

!python3 /content/gun_detection/models/research/object_detection/model_main.py \
    --pipeline_config_path={model_pipline}\
    --model_dir={model_dir} \
    --alsologtostderr \

W0911 15:14:32.664559 140538569049984 model_lib.py:771] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: None
I0911 15:14:32.664826 140538569049984 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0911 15:14:32.664959 140538569049984 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0911 15:14:32.665096 140538569049984 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0911 15:14:32.665261 140538569049984 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0911 15:14:32.665582 140538569049984 model_lib.py:787] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to_t

## Exporting The Trained model



In [ ]:


#the location where the exported model will be saved in.
output_directory = '/content/gun_detection/models/research/fine_tuned_model'

# goes through the model is the training/ dir and gets the last one.
# you could choose a specfic one instead of the last
lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')
last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)

#exports the model specifed and inference graph
!python /content/gun_detection/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={model_pipline} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

In [ ]:
#downloads the frozen model that is needed for inference
files.download(output_directory + '/frozen_inference_graph.pb')

In [ ]:
#downlaod the label map
files.download(DATA_BASE_PATH + '/label_map.pbtxt')